In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df=pd.read_csv('../input/london-bike-sharing-dataset/london_merged.csv', parse_dates=['timestamp'])
df.head()

In [3]:
df.shape

In [4]:
train=df.iloc[:17000, 1:2]
test=df.iloc[17000:17414, 1:2]

In [5]:
print(train.shape)
print(test.shape)


In [6]:
df['cnt'][:17000].plot(figsize=(15,4), legend=True)
df['cnt'][17000:].plot(figsize=(15,4), legend=True)
plt.legend(['train','test'])
plt.title('bike share demand')
plt.show()

In [7]:
from statsmodels.tsa.stattools import pacf
pacf= pacf(df['cnt'], nlags=20, method='ols')
print(pacf)

In [8]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(pacf, lags=9, method='ols', title='pa').show

In [9]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
train_scaled=sc.fit_transform(train)

In [10]:
train_scaled

In [11]:
x_train=[]
y_train=[]
for i in range(1, 17000):
    x_train.append(train_scaled[i-1:i,0])
    y_train.append(train_scaled[i,0])
    
x_train, y_train=np.array(x_train), np.array(y_train)

In [12]:
x_train.shape

In [13]:
x_train=np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 1))
x_train.shape


In [14]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN

In [15]:
rnn= Sequential()
rnn.add(SimpleRNN(activation='relu', units=6, input_shape=(1,1)))
rnn.add(Dense(activation='linear', units=1))

In [16]:
print(rnn.summary())

In [17]:
rnn.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [18]:
rnn.fit(x_train, y_train, batch_size=1, epochs=2)

In [19]:
inputs=sc.transform(test)
inputs.shape

In [20]:
x_test=[]
for i in range(1,415):
    x_test.append(inputs[i-1:i,0])
x_test=np.array(x_test)
x_test=np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
x_test.shape

In [21]:
rnn=rnn.predict(x_test)
rnn=sc.inverse_transform(rnn)

In [22]:
test1=pd.DataFrame(test)
rnn1=pd.DataFrame(rnn)

In [23]:
test1.plot(figsize=(15,4), legend=True)
plt.legend(['cnt'])
plt.title('bike share demand')
plt.show()

In [24]:
test=np.array(test)

In [25]:
plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(rnn, marker='.', label='RNN', color='red')
plt.show()

In [27]:
from keras.layers.recurrent import LSTM
lstm=Sequential()
lstm.add(LSTM(units=6, activation='relu', input_shape=(1,1)))
lstm.add(Dense(units=1, activation='linear'))

In [28]:
lstm.summary()

In [30]:
lstm.compile(loss='mse', optimizer='adam', metrics=['mse'])
lstm.fit(x_train,y_train, batch_size=1, epochs=2)

In [31]:
lstm=lstm.predict(x_test)
lstm=sc.inverse_transform(lstm)

In [32]:
plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(lstm, marker='.', label='lstm', color='green')
plt.legend()

In [57]:
plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(rnn, marker='.', label='rnn', color='red')
plt.plot(lstm, marker='.', label='lstm', color='green')
plt.legend()

In [58]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))

In [61]:
print('RNN RMSE:', RMSE(test, rnn))

In [60]:
print('LSTM RMSE:', RMSE(test, lstm))